# Parameter Tuning
* xgboost
* optuna
* GPU ver.

In [1]:
import os
import sys
import joblib
import pickle as pkl

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore')
warnings.filterwarnings(action='ignore', category=DataConversionWarning)


import pandas as pd 
import numpy as np


from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_info_columns', 500)

In [ ]:
# !git clone --recursive https://github.com/dmlc/xgboost
# cd xgboost
# !mkdir build
# cd build
# !pip install cmake --upgrade
# !cmake .. -DUSE_CUDA=ON -DUSE_NCCL=ON
# !make -j4
# cd ../python-package
# !sudo python setup.py install
# !sudo apt-get install python-setuptools
# !pip3 install scikit-learn==0.21.3 --upgrade

In [8]:
cd /content/drive/My Drive/쇼핑광고등어/3. Modeling/재빈

/content/drive/.shortcut-targets-by-id/1cWc-Nky29igOCXv9H2nYRUOItUS3kV8X/쇼핑광고등어/3. Modeling/재빈


In [5]:
!pip install shap
!pip install optuna

     |████████████████████████████████| 327kB 2.9MB/s 
  Created wheel for shap: filename=shap-0.36.0-cp36-cp36m-linux_x86_64.whl size=456449 sha256=b13e2acf360aefc2633a97a0350126a2cafff626931c69ae80399559191271e5
  Stored in directory: /root/.cache/pip/wheels/fb/15/e1/8f61106790da27e0765aaa6e664550ca2c50ea339099e799f4
Successfully built shap
     |████████████████████████████████| 235kB 2.9MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 163kB 9.0MB/s 
     |████████████████████████████████| 81kB 6.4MB/s 
     |████████████████████████████████| 81kB 6.0MB/s 
     |████████████████████████████████| 133kB 11.6MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 112kB 12.0MB/s 
  Created wheel for optuna: filename=optuna-2.1.0-cp36-none-any.whl size=321090 sha256=e23548ed82de624498530f548f6664bd824ffd295c3a3506204d9d6adf

In [9]:
import SHAPvalue as SHAP
import optimization as opt

In [10]:
data_v4 = joblib.load(os.path.join('..', '..', '1. Data', '05_분석데이터', '7th_train_FE.pkl'))
locals().update(data_v4)

In [11]:
X = data_v4["X"] ; y = data_v4["y"]
X.shape, y.shape

((35379, 402), (35379,))

In [12]:
y2 = np.log1p(y)

In [13]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
xgb_params1

{'alpha': 0.9078295349736529,
 'colsample_bylevel': 0.6765106201227887,
 'colsample_bytree': 0.5905645203230304,
 'eta': 0.006911884271921879,
 'gamma': 0.23334298443140977,
 'grow_policy': 'lossguide',
 'lambda': 0.001351665833295336,
 'max_bin': 55,
 'max_depth': 25,
 'min_child_weight': 0.06586388620533702,
 'nthread': -1,
 'num_iterations': 30000,
 'objective': 'reg:squarederror',
 'predictor': 'gpu_predictor',
 'silent': 1,
 'subsample': 0.0023791330114249945,
 'tree_method': 'gpu_hist'}

In [ ]:
%%time

xgb_mape1, xgb_pred1, xgb_shap1 = SHAP.xgb_SHAP(X, y2, xgb_params1, version='0921-xgb-optuna', cv_splits=5, scaling="MinMax", epoch=30000)

[14:51:17] WARNING: /content/xgboost/src/learner.cc:529: 
Parameters: { num_iterations, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:15.97415	valid-rmse:15.95643
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 500 rounds.
[2500]	train-rmse:0.71975	valid-rmse:0.72391
[5000]	train-rmse:0.67221	valid-rmse:0.68693
[7500]	train-rmse:0.64667	valid-rmse:0.66797
[10000]	train-rmse:0.62671	valid-rmse:0.65289
[12500]	train-rmse:0.61369	valid-rmse:0.64580
[15000]	train-rmse:0.59946	valid-rmse:0.63714
Stopping. Best iteration:
[15878]	train-rmse:0.59579	valid-rmse:0.63333

[15:44:31] WARNING: /content/xgboost/src/learner.cc:529: 
Parameters: { num_iterations, silent } might not b

KeyError: ignored

In [ ]:
# load
with open('data.pickle', 'rb') as f:
    data = pkl.load(f)

In [16]:
def objective_xgb2(trial, X, y):
    params = {
                'lambda': trial.suggest_loguniform('lambda', 1e-3, 1.0),
                'alpha': trial.suggest_loguniform('alpha', 1e-3, 1.0),
                'max_depth': trial.suggest_int('max_depth', 4, 63),
                'eta': trial.suggest_uniform('eta', 0.001, 0.01),
                'gamma': trial.suggest_loguniform('gamma', 1e-3, 1.0),
                'subsample': trial.suggest_uniform('subsample', 1e-3, 1.0), 
                'colsample_bytree': trial.suggest_uniform('colsample_bytree', 1e-3, 1.0),  
                'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 1e-3, 1.0),  
                'min_child_weight': trial.suggest_loguniform('min_child_weight', 1e-3, 15),  
                'max_bin': trial.suggest_int('max_bin', 31, 256), 
              
              
                'nthread': -1, 
                'silent' : 1 , 
                'objective': 'reg:squarederror',
                'n_estimators': 30000,
                'seed': 77
    }
    
    mape, pred = opt.xgb_model(X, y, params, cv_splits=5, scaling="MinMax", epoch=30000)
    return mape['final_mape'][0]

In [ ]:
%%time
xgb_study2 = optuna.create_study()
xgb_study2.optimize(lambda x : objective_xgb2(x, X, y2), timeout=1000, n_jobs=-1)
print(xgb_study2.best_params, xgb_study2.best_value)

[0]	train-rmse:16.00421	valid-rmse:15.98657
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 500 rounds.
[0]	train-rmse:15.98250	valid-rmse:15.96481
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 500 rounds.
[0]	train-rmse:15.98622	valid-rmse:15.96852
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 500 rounds.
[0]	train-rmse:16.05772	valid-rmse:16.04000
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 500 rounds.
[7500]	train-rmse:0.47244	valid-rmse:0.50054
[10000]	train-rmse:0.44652	valid-rmse:0.48231
[12500]	train-rmse:0.42672	valid-rmse:0.46992
[15000]	train-rmse:0.41076	valid-rmse:0.46092
[17500]	train-rmse:0.39712	valid-rmse:0.45403
[2500]	trai

In [ ]:
xgb_params2 = xgb_study2.best_params.copy()
xgb_params2['max_depth'] = int(xgb_params2['max_depth'])
xgb_params2['max_bin'] = int(xgb_params2['max_bin'])
xgb_params2['nthread'] = -1
xgb_params2['silent'] = 1
xgb_params2['objective'] = 'reg:squarederror'
xgb_params2['tree_method'] = 'gpu_hist'
xgb_params2['predictor'] = 'gpu_predictor'
xgb_params2['num_iterations'] = 30000

In [ ]:
xgb_params2

In [ ]:
%%time

xgb_mape2, xgb_pred2, xgb_shap2 = SHAP.xgb_SHAP(X, y2, xgb_params1, version='0922-xgb-optuna2', cv_splits=5, scaling="MinMax", epoch=30000)

In [ ]:
xgb_mape2

In [ ]:
xgb_params4 = {'alpha': 0.4231,
                'colsample_bylevel': 0.4011,
                'colsample_bytree': 0.7154,
                'eta': 0.008,
                'gamma': 0.5889,
                'lambda': 0.0582,
                'max_bin': 98,
                'max_depth': 12,
                'min_child_weight': 7.0562,
                'nthread': -1,
                'num_iterations': 30000,
                'objective': 'reg:squarederror',
                'predictor': 'gpu_predictor',
                'silent': 1,
                'subsample': 0.75,
                'tree_method': 'gpu_hist'}

In [ ]:
%%time

xgb_mape4, xgb_pred4, xgb_shap4 = SHAP.xgb_SHAP(X, y2, xgb_params4, version='0922-xgb-optuna', cv_splits=5, scaling="MinMax", epoch=30000)

[0]	train-rmse:15.9576	valid-rmse:15.9399
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 500 rounds.
[2500]	train-rmse:0.157558	valid-rmse:0.419415
Stopping. Best iteration:
[2599]	train-rmse:0.154696	valid-rmse:0.419361

